In [1]:
!pip install pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=e1e1b2733f0eff4092dafc882ee38ce5265c32047a11832087a3d425bbeb4ba5
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


In [2]:
!pip install xlrd

In [3]:
import pandas as pd
import pandasql as ps
#
#Importing customer dataset
customer_dim = pd.read_excel("https://github.com/ItsAwSM/SQL-for-Data-Analytics/blob/main/sales_database.xlsx?raw=true", 
                             sheet_name="customer_dim")
#Importing product dataset
product_dim = pd.read_excel("https://github.com/ItsAwSM/SQL-for-Data-Analytics/blob/main/sales_database.xlsx?raw=true", 
                            sheet_name="product_dim")
#Importing delivery person dataset
delivery_person_dim = pd.read_excel("https://github.com/ItsAwSM/SQL-for-Data-Analytics/blob/main/sales_database.xlsx?raw=true",
                                    sheet_name="delivery_person_dim")
#Importing pincode dataset
pincode_dim = pd.read_excel("https://github.com/ItsAwSM/SQL-for-Data-Analytics/blob/main/sales_database.xlsx?raw=true", 
                            sheet_name="pincode_dim")
#Importing order dataset
order_dim = pd.read_excel("https://github.com/ItsAwSM/SQL-for-Data-Analytics/blob/main/sales_database.xlsx?raw=true", 
                          sheet_name="order_dim")

In [26]:
customer_dim.head()

,cust_id,first_name,last_name,email,phone,primary_pincode,gender,dob,joining_date
0,10000001,Rahul,Gupta,rahul.gupta@gmail.com,8960722319,110001,male,02-12-1977,19-02-2016
1,10000002,Abhinav,Sharma,abhinav.sharma@gmail.com,8857852863,400001,male,20-09-1995,01-10-2016
2,10000003,Anubhav,Patel,anubhav.patel@gmail.com,8492146048,560001,male,26-12-1986,16-08-2018
3,10000004,Neha,Verma,neha.verma@gmail.com,8631861914,600001,female,20-01-2002,28-07-2009
4,10000005,John,Bernard,john.bernard@gmail.com,8215391844,500001,male,NaN,05-03-2017


In [27]:
product_dim.head()

,product_id,product_name,brand,category,procurement_cost_per_unit,mrp
0,12345,HP XYZ Mouse,HP,mouse,650,1500
1,12346,HP 241H,HP,laptop,45000,80000
2,12347,HP 8GB Pendrive,HP,pendrive,350,800
3,12348,Dell ABC Mouse,Dell,mouse,350,1100
4,12349,Dell AX420,Dell,laptop,30000,75000


In [28]:
delivery_person_dim.head()

,delivery_person_id,name,joining_date,pincode
0,1000008,Indranoor Singh,08-03-2007,110001
1,1000002,Simon Williams,23-02-2007,400001
2,1000004,Anubhav Tyagi,16-11-2007,400001
3,1000006,Aviral Vats,16-11-2007,400001
4,1000009,Rocky Tiwari,16-11-2007,400001


In [29]:
pincode_dim.head()

,pincode,city,state
0,110001,New Delhi,New Delhi
1,400001,Mumbai,Maharastra
2,560001,Bangalore,Karnataka
3,600001,Chennai,Tamil Nadu
4,500001,Hyderabad,Telangana


In [30]:
order_dim.head()

,order_id,order_type,cust_id,order_date,delivery_date,tot_units,displayed_selling_price_per_unit,total_amount_paid,product_id,delivery_person_id,payment_type,delivery_pincode
0,10000000001,buy,10000009,01-01-2020,03-01-2020,2,73500,138180,12349,1000003,cash,500001
1,10000000002,buy,10000004,01-01-2020,04-01-2020,9,67500,577125,12349,1000005,cash,700001
2,10000000003,buy,10000012,01-01-2020,02-01-2020,2,60750,119070,12349,1000002,netbanking,400001
3,10000000004,buy,10000012,01-01-2020,03-01-2020,8,736,5824,12347,1000010,cash,600001
4,10000000005,buy,10000008,01-01-2020,06-01-2020,1,60750,57105,12349,1000003,debit,500001


In [4]:
# Q0. How many customers are male and how many are female?
#
ans = ps.sqldf("""

SELECT 
    c.gender,
    COUNT(c.cust_id) AS cust_cnt
FROM customer_dim AS c
GROUP BY c.gender

""")

ans

,gender,cust_cnt
0,female,4
1,male,10


In [5]:
# Q1. How many customers do not have DOB information available?
#
ans = ps.sqldf("""

SELECT 
    COUNT(c.cust_id) AS cust_cnt
FROM customer_dim AS c
WHERE c.dob IS NULL

""")

ans

,cust_cnt
0,2


In [6]:
# Q2. How many customers are there in each pincode and gender combination?
#
ans = ps.sqldf("""

SELECT 
    c.primary_pincode,
    c.gender,
    COUNT(c.cust_id) AS cust_cnt
FROM customer_dim AS c
GROUP BY 
    c.primary_pincode,
    c.gender

""")

ans

,primary_pincode,gender,cust_cnt
0,110001,male,3
1,400001,male,2
2,500001,female,1
3,500001,male,1
4,560001,female,1
5,560001,male,1
6,600001,female,1
7,600001,male,1
8,700001,female,1
9,700001,male,2


In [7]:
# Q3. Print product name and mrp for products which have more than 50000 MRP
#
ans = ps.sqldf("""

SELECT 
    p.product_name,
    p.mrp
FROM product_dim AS p
WHERE p.mrp>50000

""")

ans

,product_name,mrp
0,HP 241H,80000
1,Dell AX420,75000


In [8]:
# Q4. How many delivery personal are there in each pincode?
#
ans = ps.sqldf("""

SELECT 
    d.pincode,
    COUNT(d.delivery_person_id) AS dp_cnt
FROM delivery_person_dim AS d
GROUP BY d.pincode


""")

ans

,pincode,dp_cnt
0,110001,1
1,400001,4
2,500001,1
3,560001,1
4,600001,1
5,700001,2


In [9]:
#Q5. For each Pin code, print the count of orders, sum of total amount paid, average amount paid, maximum amount paid, minimum amount paid for the 
#transactions which were paid by 'cash'. Take only 'buy' order types
#
ans = ps.sqldf("""

SELECT
    o.delivery_pincode,
    COUNT(o.order_id) AS order_cnt,
    SUM(o.total_amount_paid) AS total_amount_paid,
    AVG(o.total_amount_paid) AS avg_amount_paid,
    MAX(o.total_amount_paid) AS max_amount_paid,
    MIN(o.total_amount_paid) AS min_amount_paid
FROM order_dim as o
WHERE o.payment_type = 'cash'
    AND o.order_type = 'buy'
GROUP BY 
    o.delivery_pincode

""")

ans

,delivery_pincode,order_cnt,total_amount_paid,avg_amount_paid,max_amount_paid,min_amount_paid
0,110001,19,4026734,211933.368421,608103,676
1,400001,105,11546300,109964.761905,669750,644
2,500001,28,4798422,171372.214286,646800,1314
3,560001,19,2829381,148914.789474,609120,662
4,600001,19,1456296,76647.157895,669600,1213
5,700001,53,6871936,129659.169811,721280,687


In [10]:
#Q6. For each delivery_person_id, print the count of orders and total amount paid for product_id = 12350 or 12348 and total units > 8. Sort the output by 
#total amount paid in descending order. Take only 'buy' order types

ans = ps.sqldf("""

SELECT
    o.delivery_person_id,
    COUNT(o.order_id) AS order_cnt,
    SUM(o.total_amount_paid) AS total_amount_paid
FROM order_dim as o
WHERE o.product_id IN (12350,12348)
    AND o.tot_units > 8
    AND o.order_type = 'buy'
GROUP BY 
    o.delivery_person_id
ORDER BY 
    SUM(o.total_amount_paid) DESC

""")

ans

,delivery_person_id,order_cnt,total_amount_paid
0,1000002,10,76801
1,1000010,7,56285
2,1000003,6,52828
3,1000001,6,51653
4,1000009,6,49142
5,1000008,6,48424
6,1000005,5,43677
7,1000007,5,41535
8,1000004,5,35452
9,1000006,4,32915


In [11]:
#Q7. Print the Full names (first name plus last name) for customers that have email on "gmail.com"?

ans = ps.sqldf("""

SELECT 
    c.first_name || ' ' || c.last_name AS name
FROM customer_dim AS c
WHERE c.email LIKE '%gmail%'

""")

ans

,name
0,Rahul Gupta
1,Abhinav Sharma
2,Anubhav Patel
3,Neha Verma
4,John Bernard
5,Muhammad Ali
6,Ahmed Khan
7,Paras Rana


In [12]:
#Q8. How many orders had #units between 1-3, 4-6 and 7+? Take only 'buy' order types

ans = ps.sqldf("""

SELECT 
    CASE 
        WHEN o.tot_units <= 3 THEN '1. 1-3'
        WHEN o.tot_units <= 6 THEN '2. 4-6'
    ELSE '3. 7+' END AS unit_bkt,
    COUNT(o.order_id) AS order_cnt
FROM order_dim AS o
WHERE o.order_type = 'buy'
GROUP BY 
    CASE 
        WHEN o.tot_units <= 3 THEN '1. 1-3'
        WHEN o.tot_units <= 6 THEN '2. 4-6'
    ELSE '3. 7+' END    

""")

ans

,unit_bkt,order_cnt
0,1. 1-3,314
1,2. 4-6,314
2,3. 7+,372


In [13]:
#Q9. Which pincode has average amount paid more than 150,000? Take only 'buy' order types

ans = ps.sqldf("""

SELECT
    o.delivery_pincode,
    AVG(total_amount_paid) AS avg_amount_paid
FROM order_dim AS o
WHERE o.order_type = 'buy'
GROUP BY o.delivery_pincode
HAVING AVG(total_amount_paid)>150000

""")

ans

,delivery_pincode,avg_amount_paid
0,110001,158145.650485


In [14]:
#Q10. Create following columns from order_dim data -

#order_date
#Order day
#Order month
#Order year

ans = ps.sqldf("""

SELECT 
    o.order_date,
    SUBSTR(o.order_date, 1, 2) AS order_day,
    SUBSTR(o.order_date, 4, 2) AS order_month,
    SUBSTR(o.order_date, 7, 4) AS order_year
FROM order_dim AS o
WHERE o.order_type = 'buy'

""")

#printing random 5 rows only
ans.sample(5)

,order_date,order_day,order_month,order_year
683,04-07-2020,04,07,2020
378,14-04-2020,14,04,2020
450,02-05-2020,02,05,2020
817,12-08-2020,12,08,2020
864,25-08-2020,25,08,2020


In [15]:
#Q11. How many total orders were there in each month and how many of them were returned? Add a column for return rate too.
#return rate = (100.0 * total return orders) / total buy orders
#Hint: You will need to combine SUM() with CASE WHEN

ans = ps.sqldf("""

SELECT 
    SUBSTR(o.order_date, 4, 2) AS order_month,
    SUM(CASE WHEN o.order_type = 'buy' THEN 1 ELSE 0 END) AS tot_buy_orders,
    SUM(CASE WHEN o.order_type = 'return' THEN 1 ELSE 0 END) AS tot_return_orders,
    100.0*SUM(CASE WHEN o.order_type = 'return' THEN 1 ELSE 0 END)/SUM(CASE WHEN o.order_type = 'buy' THEN 1 ELSE 0 END) AS return_rate
FROM order_dim AS o
GROUP BY SUBSTR(o.order_date, 4, 2)

""")

ans

,order_month,tot_buy_orders,tot_return_orders,return_rate
0,01,119,3,2.521008
1,02,107,7,6.542056
2,03,103,6,5.825243
3,04,115,6,5.217391
4,05,117,8,6.837607
5,06,106,3,2.830189
6,07,110,4,3.636364
7,08,109,5,4.587156
8,09,109,5,4.587156
9,10,5,3,60.000000


In [16]:
#Q12. How many units have been sold by each brand? Also get total returned units for each brand.

ans = ps.sqldf("""

SELECT 
    p.brand,
    SUM(CASE WHEN o.order_type = 'buy' THEN o.tot_units ELSE 0 END) AS total_sold,
    SUM(CASE WHEN o.order_type = 'return' THEN o.tot_units ELSE 0 END) AS total_returned
FROM product_dim AS p
LEFT JOIN order_dim AS o
    ON p.product_id = o.product_id
GROUP BY p.brand

""")

ans

,brand,total_sold,total_returned
0,Dell,2701,112
1,HP,2666,145


In [17]:
#Q13. How many distinct customers and delivery boys are there in each state?

ans = ps.sqldf("""

SELECT 
    pin.state,
    COUNT(DISTINCT c.cust_id) AS cust_cnt,
    COUNT(DISTINCT d.delivery_person_id) AS dc_cnt
FROM pincode_dim AS pin

LEFT JOIN customer_dim AS c
    ON pin.pincode = c.primary_pincode

LEFT JOIN delivery_person_dim AS d
    ON pin.pincode = d.pincode
    
GROUP BY pin.state

""")

ans

,state,cust_cnt,dc_cnt
0,Karnataka,2,1
1,Maharastra,2,4
2,New Delhi,3,1
3,Tamil Nadu,2,1
4,Telangana,2,1
5,West Bengal,3,2


In [18]:
#Q14. For every customer, print how many total units were ordered, how many units were ordered from their primary_pincode and how many were ordered not from 
#the primary_pincode. Also calulate the percentage of total units which were ordered from primary_pincode(remember to multiply the numerator by 100.0). 
#Sort by the percentage column in descending order.

ans = ps.sqldf("""

SELECT 
    c.cust_id,
    SUM(o.tot_units) AS tot_units,
    SUM(CASE WHEN c.primary_pincode = o.delivery_pincode THEN o.tot_units ELSE 0 END) AS units_same_city,
    SUM(CASE WHEN c.primary_pincode != o.delivery_pincode THEN o.tot_units ELSE 0 END) AS units_diff_city,
    100.0 * SUM(CASE WHEN c.primary_pincode = o.delivery_pincode THEN o.tot_units ELSE 0 END)/SUM(o.tot_units) AS perc_same_city
FROM customer_dim AS c
LEFT JOIN order_dim AS o
    ON c.cust_id = o.cust_id
GROUP BY c.cust_id
ORDER BY perc_same_city DESC

""")

ans

,cust_id,tot_units,units_same_city,units_diff_city,perc_same_city
0,10000002,372,164,208,44.086022
1,10000008,410,152,258,37.073171
2,10000012,534,109,425,20.411985
3,10000007,369,72,297,19.512195
4,10000005,375,59,316,15.733333
5,10000006,290,44,246,15.172414
6,10000003,413,61,352,14.769976
7,10000009,537,66,471,12.290503
8,10000004,398,48,350,12.060302
9,10000014,353,42,311,11.898017


In [20]:
#Q15. For each product name, print the sum of number of units, total amount paid, total displayed selling price, total mrp of these units, and finally 
#the net discount from selling price (i.e. 100.0 - 100.0 total amount paid / total displayed selling price) AND the net discount from mrp
# (i.e. 100.0 - 100.0 total amount paid / total mrp)

ans = ps.sqldf("""

SELECT 
    p.product_name,
    SUM(o.tot_units) AS tot_units,
    SUM(o.total_amount_paid) AS total_amount_paid,
    SUM(o.displayed_selling_price_per_unit * o.tot_units) AS total_displayed_selling_price,
    SUM(p.mrp * o.tot_units) AS total_mrp,
    100.0 - 100.0 * SUM(o.total_amount_paid)/SUM(o.displayed_selling_price_per_unit * o.tot_units) AS discount_from_sp,
    100.0 - 100.0 * SUM(o.total_amount_paid)/SUM(p.mrp * o.tot_units) AS discount_from_mrp
FROM product_dim AS p
LEFT JOIN order_dim AS o
    ON p.product_id = o.product_id
GROUP BY p.product_name

""")

ans

,product_name,tot_units,total_amount_paid,total_displayed_selling_price,total_mrp,discount_from_sp,discount_from_mrp
0,Dell 8GB Pendrive,889,574506,670592,755650,14.328534,23.971945
1,Dell ABC Mouse,942,809662,928510,1036200,12.799862,21.862382
2,Dell AX420,982,58124196,65829000,73650000,11.704270,21.080521
3,HP 241H,884,51396664,63275200,70720000,18.772815,27.323722
4,HP 8GB Pendrive,904,578605,654288,723200,11.567230,19.993778
5,HP XYZ Mouse,1023,1155504,1372470,1534500,15.808433,24.698338


In [21]:
#Q16. For every order_id (exclude returns), get the product name and calculate the discount percentage from selling price. Sort by highest discount and 
#print only those rows where discount percentage was above 10.10%.

ans = ps.sqldf("""

SELECT 
    o.order_id,
    p.product_name,
    100.0 - 100.0 * o.total_amount_paid/(o.displayed_selling_price_per_unit * o.tot_units) AS discount_from_sp
FROM product_dim AS p
LEFT JOIN order_dim AS o
    ON p.product_id = o.product_id
WHERE o.order_type = 'buy'
GROUP BY 
    o.order_id,
    p.product_name
HAVING discount_from_sp>10.10

""")

ans

,order_id,product_name,discount_from_sp
0,10000000133,Dell 8GB Pendrive,10.123119
1,10000000150,Dell 8GB Pendrive,10.107015
2,10000000155,Dell 8GB Pendrive,10.123119
3,10000000160,HP 8GB Pendrive,10.119048
4,10000000409,HP 8GB Pendrive,10.106383
5,10000000575,HP 8GB Pendrive,10.101010
6,10000000580,Dell 8GB Pendrive,10.110803
7,10000000653,HP 8GB Pendrive,10.119048
8,10000000902,Dell ABC Mouse,10.101010
9,10000000997,Dell 8GB Pendrive,10.102302


In [22]:
#Q17. Using the per unit procurement cost in product_dim, find which product category has made the most profit in both absolute amount and percentage
#Absolute Profit = Total Amt Sold - Total Procurement Cost
#Percentage Profit = 100.0 * Total Amt Sold / Total Procurement Cost - 100.0

ans = ps.sqldf("""

SELECT 
    p.category,
    SUM(o.total_amount_paid) - SUM(p.procurement_cost_per_unit * o.tot_units) AS abs_profit,
    100.0 * SUM(o.total_amount_paid)/SUM(p.procurement_cost_per_unit * o.tot_units) - 1 AS perc_profit
FROM product_dim AS p
LEFT JOIN order_dim AS o
    ON p.product_id = o.product_id
GROUP BY 
    p.category

""")

ans

,category,abs_profit,perc_profit
0,laptop,40280860,157.175708
1,mouse,970516,196.573619
2,pendrive,614461,213.074260


In [23]:
#Q18. For every delivery person(use their name), print the total number of order ids (exclude returns) by month in seperate columns i.e. there should be 
#one row for each delivery_person_id and 12 columns for every month in the year

ans = ps.sqldf("""

SELECT 
    d.name,
    SUM(CASE WHEN SUBSTR(o.order_date, 4, 2) = '01' THEN 1 ELSE 0 END) AS Jan,
    SUM(CASE WHEN SUBSTR(o.order_date, 4, 2) = '02' THEN 1 ELSE 0 END) AS Feb,
    SUM(CASE WHEN SUBSTR(o.order_date, 4, 2) = '03' THEN 1 ELSE 0 END) AS Mar,
    SUM(CASE WHEN SUBSTR(o.order_date, 4, 2) = '04' THEN 1 ELSE 0 END) AS Apr,
    SUM(CASE WHEN SUBSTR(o.order_date, 4, 2) = '05' THEN 1 ELSE 0 END) AS May,
    SUM(CASE WHEN SUBSTR(o.order_date, 4, 2) = '06' THEN 1 ELSE 0 END) AS Jun,
    SUM(CASE WHEN SUBSTR(o.order_date, 4, 2) = '07' THEN 1 ELSE 0 END) AS Jul,
    SUM(CASE WHEN SUBSTR(o.order_date, 4, 2) = '08' THEN 1 ELSE 0 END) AS Aug,
    SUM(CASE WHEN SUBSTR(o.order_date, 4, 2) = '09' THEN 1 ELSE 0 END) AS Sep,
    SUM(CASE WHEN SUBSTR(o.order_date, 4, 2) = '10' THEN 1 ELSE 0 END) AS Oct,
    SUM(CASE WHEN SUBSTR(o.order_date, 4, 2) = '11' THEN 1 ELSE 0 END) AS Nov,
    SUM(CASE WHEN SUBSTR(o.order_date, 4, 2) = '12' THEN 1 ELSE 0 END) AS Dec
FROM delivery_person_dim AS d
INNER JOIN order_dim AS o
    ON d.delivery_person_id = o.delivery_person_id 
WHERE o.order_type = 'buy'
GROUP BY 
    d.name

""")

ans

,name,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,Anubhav Tyagi,6,12,15,8,16,14,10,6,13,0,0,0
1,Aviral Vats,11,12,5,15,17,14,10,12,11,1,0,0
2,Indranoor Singh,17,13,9,11,16,15,7,6,9,0,0,0
3,Qutub Mohammad,7,9,9,13,9,5,12,5,14,1,0,0
4,Robert Langdon,19,12,11,12,15,6,14,13,6,0,0,0
5,Rocky Tiwari,10,11,13,13,4,10,12,19,10,0,0,0
6,Sanjay Krishnan,8,19,10,13,7,6,8,12,14,0,0,0
7,Saurabh Ganguly,13,4,16,13,7,12,12,12,12,2,0,0
8,Simon Williams,9,5,8,10,14,13,14,12,10,0,0,0
9,Vivek Gandhi,19,10,7,7,12,11,11,12,10,1,0,0


In [24]:
#Q19. For each gender - male and female - find the absolute and percentage profit (like in Q16) by product name.

ans = ps.sqldf("""

SELECT 
    c.gender,
    p.category,
    SUM(o.total_amount_paid) - SUM(p.procurement_cost_per_unit * o.tot_units) AS abs_profit,
    100.0 * SUM(o.total_amount_paid)/SUM(p.procurement_cost_per_unit * o.tot_units) - 1 AS perc_profit
FROM customer_dim AS c

LEFT JOIN order_dim AS o
    ON c.cust_id = o.cust_id
    
LEFT JOIN product_dim AS p
    ON p.product_id = o.product_id
    
GROUP BY 
    c.gender,
    p.category

""")

ans

,gender,category,abs_profit,perc_profit
0,female,laptop,12452614,154.308079
1,female,mouse,207256,177.819547
2,female,pendrive,165026,205.502743
3,male,laptop,27828246,158.557509
4,male,mouse,763260,203.313243
5,male,pendrive,449435,216.131874


In [25]:
#Q20. Generally the more numbers of units you buy, the more discount seller will give you. For 'Dell AX420' is there a relationship between number of 
#units ordered and average discount from selling price? Take only 'buy' order types

ans = ps.sqldf("""

SELECT 
    o.tot_units,
    COUNT(order_id) AS total_orders,
    100.0 - 100.0 * o.total_amount_paid/(o.displayed_selling_price_per_unit * o.tot_units) AS discount_from_sp
FROM product_dim AS p
LEFT JOIN order_dim AS o
    ON p.product_id = o.product_id
WHERE o.order_type = 'buy'
    AND p.product_name = 'Dell AX420'
GROUP BY 
    o.tot_units

""")

ans

,tot_units,total_orders,discount_from_sp
0,1,21,3.000766
1,2,16,1.000687
2,3,19,1.000000
3,4,16,8.000000
4,5,19,7.000784
5,6,16,2.000000
6,7,18,10.000000
7,8,16,4.000000
8,9,19,6.000000
9,10,16,2.000000
